# Mixed Logit example

Using the xlogit package in Python

## Swiss Metro dataset (from vignette)

In [1]:
from xlogit import MixedLogit
import pandas as pd
import numpy as np

MixedLogit.check_if_gpu_available()


df_wide = pd.read_table("http://transp-or.epfl.ch/data/swissmetro.dat", sep='\t')

# Keep only observations for commute and business purposes that contain known choices
df_wide = df_wide[(df_wide['PURPOSE'].isin([1, 3]) & (df_wide['CHOICE'] != 0))]

df_wide['custom_id'] = np.arange(len(df_wide))  # Add unique identifier
df_wide['CHOICE'] = df_wide['CHOICE'].map({1: 'TRAIN', 2:'SM', 3: 'CAR'})
df_wide

*** No GPU device found. Verify CuPy is properly installed


,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE,custom_id
0,2,0,1,1,1,0,1,1,0,3,...,48,120,63,52,20,0,117,65,SM,0
1,2,0,1,1,1,0,1,1,0,3,...,48,30,60,49,10,0,117,84,SM,1
2,2,0,1,1,1,0,1,1,0,3,...,48,60,67,58,30,0,117,52,SM,2
3,2,0,1,1,1,0,1,1,0,3,...,40,30,63,52,20,0,72,52,SM,3
4,2,0,1,1,1,0,1,1,0,3,...,36,60,63,42,20,0,90,84,SM,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8446,3,1,1,939,3,1,7,3,1,5,...,13,30,50,17,30,0,130,64,TRAIN,6763
8447,3,1,1,939,3,1,7,3,1,5,...,12,30,53,16,10,0,80,80,TRAIN,6764
8448,3,1,1,939,3,1,7,3,1,5,...,16,60,50,16,20,0,80,64,TRAIN,6765
8449,3,1,1,939,3,1,7,3,1,5,...,16,30,53,17,30,0,80,104,TRAIN,6766


In [2]:
from xlogit.utils import wide_to_long

df = wide_to_long(df_wide, id_col='custom_id', alt_name='alt', sep='_',
                  alt_list=['TRAIN', 'SM', 'CAR'], empty_val=0,
                  varying=['TT', 'CO', 'HE', 'AV', 'SEATS'], alt_is_prefix=True)
df

,custom_id,alt,TT,CO,HE,AV,SEATS,GROUP,SURVEY,SP,...,TICKET,WHO,LUGGAGE,AGE,MALE,INCOME,GA,ORIGIN,DEST,CHOICE
0,0,TRAIN,112,48,120,1,0,2,0,1,...,1,1,0,3,0,2,0,2,1,SM
1,0,SM,63,52,20,1,0,2,0,1,...,1,1,0,3,0,2,0,2,1,SM
2,0,CAR,117,65,0,1,0,2,0,1,...,1,1,0,3,0,2,0,2,1,SM
3,1,TRAIN,103,48,30,1,0,2,0,1,...,1,1,0,3,0,2,0,2,1,SM
4,1,SM,60,49,10,1,0,2,0,1,...,1,1,0,3,0,2,0,2,1,SM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20299,6766,SM,53,17,30,1,0,3,1,1,...,7,3,1,5,1,2,0,1,2,TRAIN
20300,6766,CAR,80,104,0,1,0,3,1,1,...,7,3,1,5,1,2,0,1,2,TRAIN
20301,6767,TRAIN,108,13,60,1,0,3,1,1,...,7,3,1,5,1,2,0,1,2,TRAIN
20302,6767,SM,53,21,30,1,0,3,1,1,...,7,3,1,5,1,2,0,1,2,TRAIN


In [3]:
df['ASC_TRAIN'] = np.ones(len(df))*(df['alt'] == 'TRAIN')
df['ASC_CAR'] = np.ones(len(df))*(df['alt'] == 'CAR')
df['TT'], df['CO'] = df['TT']/100, df['CO']/100  # Scale variables
annual_pass = (df['GA'] == 1) & (df['alt'].isin(['TRAIN', 'SM']))
df.loc[annual_pass, 'CO'] = 0  # Cost zero for pass holders
df

,custom_id,alt,TT,CO,HE,AV,SEATS,GROUP,SURVEY,SP,...,LUGGAGE,AGE,MALE,INCOME,GA,ORIGIN,DEST,CHOICE,ASC_TRAIN,ASC_CAR
0,0,TRAIN,1.12,0.48,120,1,0,2,0,1,...,0,3,0,2,0,2,1,SM,1.0,0.0
1,0,SM,0.63,0.52,20,1,0,2,0,1,...,0,3,0,2,0,2,1,SM,0.0,0.0
2,0,CAR,1.17,0.65,0,1,0,2,0,1,...,0,3,0,2,0,2,1,SM,0.0,1.0
3,1,TRAIN,1.03,0.48,30,1,0,2,0,1,...,0,3,0,2,0,2,1,SM,1.0,0.0
4,1,SM,0.60,0.49,10,1,0,2,0,1,...,0,3,0,2,0,2,1,SM,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20299,6766,SM,0.53,0.17,30,1,0,3,1,1,...,1,5,1,2,0,1,2,TRAIN,0.0,0.0
20300,6766,CAR,0.80,1.04,0,1,0,3,1,1,...,1,5,1,2,0,1,2,TRAIN,0.0,1.0
20301,6767,TRAIN,1.08,0.13,60,1,0,3,1,1,...,1,5,1,2,0,1,2,TRAIN,1.0,0.0
20302,6767,SM,0.53,0.21,30,1,0,3,1,1,...,1,5,1,2,0,1,2,TRAIN,0.0,0.0


In [4]:
varnames=['ASC_CAR', 'ASC_TRAIN', 'CO', 'TT']
model = MixedLogit()
model.fit(X=df[varnames], y=df['CHOICE'], varnames=varnames,
          alts=df['alt'], ids=df['custom_id'], avail=df['AV'],
          panels=df["ID"], randvars={'TT': 'n'}, n_draws=1500,
          optim_method='L-BFGS-B')
model.summary()

Optimization terminated successfully.
    Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
    Iterations: 14
    Function evaluations: 15
Estimation time= 66.0 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
ASC_CAR                 0.2831085     0.0560480     5.0511798      4.51e-07 ***
ASC_TRAIN              -0.5722790     0.0794780    -7.2004736      6.65e-13 ***
CO                     -1.6601703     0.0778870   -21.3151011      1.26e-97 ***
TT                     -3.2289850     0.1749806   -18.4533834      3.17e-74 ***
sd.TT                   3.6485337     0.1683459    21.6728357     9.26e-101 ***
---------------------------------------------------------------------------
Significance:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Log-Likelihood= -4359.218
AIC= 8728.436
BIC= 8762.536


## Electricity dataset (from vignette)

In [6]:
df = pd.read_csv("https://raw.github.com/arteagac/xlogit/master/examples/data/electricity_long.csv")
df

,choice,id,alt,pf,cl,loc,wk,tod,seas,chid
0,0,1,1,7,5,0,1,0,0,1
1,0,1,2,9,1,1,0,0,0,1
2,0,1,3,0,0,0,0,0,1,1
3,1,1,4,0,5,0,1,1,0,1
4,0,1,1,7,0,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...
17227,0,361,4,0,1,1,0,0,1,4307
17228,1,361,1,9,0,0,1,0,0,4308
17229,0,361,2,7,0,0,0,0,0,4308
17230,0,361,3,0,1,0,1,0,1,4308


In [7]:
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
model = MixedLogit()
model.fit(X=df[varnames],
          y=df['choice'],
          varnames=varnames,
          ids=df['chid'],
          panels=df['id'],
          alts=df['alt'],
          n_draws=600,
          randvars={'pf': 'n', 'cl': 'n', 'loc': 'n',
                    'wk': 'n', 'tod': 'n', 'seas': 'n'})
model.summary()

Optimization terminated successfully.
    Message: The gradients are close to zero
    Iterations: 25
    Function evaluations: 27
Estimation time= 28.5 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -0.9972102     0.0361699   -27.5701452     3.07e-154 ***
cl                     -0.2196812     0.0143577   -15.3006166      1.67e-51 ***
loc                     2.2901807     0.0891457    25.6903002     1.51e-135 ***
wk                      1.6943247     0.0719012    23.5646304     1.37e-115 ***
tod                    -9.6752279     0.3117174   -31.0384572     4.54e-191 ***
seas                   -9.6961836     0.3111496   -31.1624480     1.94e-192 ***
sd.pf                   0.2207255     0.0126686    17.4230731      9.44e-66 ***
sd.cl                   0.4115547     0.0200678    

## Fishing dataset (from vignette)

In [9]:
df = pd.read_csv("https://raw.github.com/arteagac/xlogit/master/examples/data/fishing_long.csv")
varnames = ['price', 'catch']
model = MixedLogit()
model.fit(X=df[varnames],
          y=df['choice'],
          varnames=varnames,
          alts=df['alt'],
          ids=df['id'],
          n_draws=1000,
          randvars={'price': 'n', 'catch': 'n'})
model.summary()

Optimization terminated successfully.
    Message: The gradients are close to zero
    Iterations: 19
    Function evaluations: 31
Estimation time= 6.8 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
price                  -0.0272460     0.0024982   -10.9062938      1.87e-26 ***
catch                   1.3271150     0.1724869     7.6940046      2.99e-14 ***
sd.price                0.0102130     0.0022025     4.6370078      3.93e-06 ***
sd.catch               -1.5706825     0.5732798    -2.7398182       0.00624 ** 
---------------------------------------------------------------------------
Significance:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Log-Likelihood= -1300.511
AIC= 2609.023
BIC= 2629.323


Predictions

In [10]:
choices, freq = model.predict(X=df[varnames], varnames=varnames, ids=df['id'],
                              alts=df['alt'], return_freq=True)
print(f"base: {freq}")

df.loc[df['alt']=='boat', 'price'] *= 1.2  # 20 percent price increase
choices, freq = model.predict(X=df[varnames], varnames=varnames, ids=df['id'],
                              alts=df['alt'], return_freq=True)
print(f"updated: {freq}")

base: {'beach': 0.223, 'boat': 0.461, 'charter': 0.228, 'pier': 0.089}
updated: {'beach': 0.238, 'boat': 0.379, 'charter': 0.278, 'pier': 0.105}


## Car dataset (from vignette)

In [11]:
df = pd.read_csv("https://raw.github.com/arteagac/xlogit/master/examples/data/car100_long.csv")
df['price'] = -df['price']/10000
df['opcost'] = -df['opcost']
varnames = ['hiperf', 'medhiperf', 'price', 'opcost', 'range', 'ev', 'hybrid']
model = MixedLogit()
model.fit(X=df[varnames],
          y=df['choice'],
          varnames=varnames,
          alts=df['alt'],
          ids=df['choice_id'],
          panels=df['person_id'],
          randvars = {'price': 'ln', 'opcost': 'n',
                      'range': 'ln', 'ev':'n', 'hybrid': 'n'},
          n_draws = 100)
model.summary()

Optimization terminated successfully.
    Message: The gradients are close to zero
    Iterations: 31
    Function evaluations: 34
Estimation time= 2.0 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
hiperf                  0.1058410     0.0971974     1.0889290         0.276    
medhiperf               0.5604997     0.0977352     5.7348796      1.18e-08 ***
price                  -0.7871346     0.1048151    -7.5097475      1.02e-13 ***
opcost                  0.0110846     0.0041762     2.6542027       0.00803 ** 
range                  -0.6857957     0.4362769    -1.5719276         0.116    
ev                     -1.5574339     0.3250817    -4.7908996      1.83e-06 ***
hybrid                  0.6883966     0.1467451     4.6911049      2.97e-06 ***
sd.price                0.8666443     0.0926631     